In [1]:
!pip install PySastrawi

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import json
import pickle
from tensorflow.keras.callbacks import EarlyStopping
import nltk
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, callbacks, optimizers
import numpy as np

In [4]:
# Load the intents file
with open('nlp_dataset_faq.json') as data_file:
    intents = json.load(data_file)

In [5]:
import nltk
nltk.download('punkt')

pattern = "Apa kabar?"
w = nltk.word_tokenize(pattern.lower())
print(w)


['apa', 'kabar', '?']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Download the 'punkt_tab' data package
nltk.download('punkt_tab')

# Pre-processing
words = []
classes = []
documents = []
ignore = ['','!', '"', "'", '(', ')', ',', '-', '.', ':', ';','[', ']', '_','?','.'
          'adalah', 'akan', 'aku', 'anda', 'atau', 'dalam', 'dan',
          'dari', 'dengan', 'di', 'dia', 'harus', 'ini', 'itu', 'jika', 'kami', 'kamu', 'ke',
          'kita', 'mereka', 'oleh', 'pada', 'saya', 'sebuah', 'sedang', 'sementara', 'tanpa',
          'tapi', 'telah', 'untuk', 'yang', '{', '}', 'merasa']



#tokenize
for intent in intents:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern.lower())  # Tokenizing
        words.extend([word for word in w if word not in ignore])  #Ignore
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
# Melakukan stemming dan normalisasi data
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# Menghapus class duplikat dengan 'set'
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

1634 documents
81 classes ['apa_itu_gangguan_mental', 'badmood_dampak', 'badmood_efek_jangka_panjang', 'badmood_gejala', 'badmood_pengobatan', 'badmood_penyebab', 'badmood_preventif', 'badmood_solusi_pribadi', 'badmood_terhadap_kesehatan_fisik', 'badmood_terhadap_pekerjaan', 'bunuh_diri', 'cara_mengatasi_benci', 'cara_mengatasi_bingung', 'cara_mengatasi_cemas', 'cara_mengatasi_cemburu', 'cara_mengatasi_dendam', 'cara_mengatasi_diremehkan', 'cara_mengatasi_frustasi', 'cara_mengatasi_gangguan_mental', 'cara_mengatasi_hampa', 'cara_mengatasi_iri', 'cara_mengatasi_kecewa', 'cara_mengatasi_kesal', 'cara_mengatasi_malu', 'cara_mengatasi_marah', 'cara_mengatasi_merasa_sendiri', 'cara_mengatasi_overthinking', 'cara_mengatasi_rasa_bersalah', 'cara_mengatasi_sedih', 'cara_mengatasi_takut', 'cara_mengatasi_takut_gagal', 'cara_mengatasi_tidak_dihargai', 'cara_positive_thinking', 'diagnosis_gangguan_mental', 'dukungan_mental_terapi', 'efek_gangguan_mental', 'emosi_bingung', 'emosi_frustrasi', 'emos

In [8]:
import random
import numpy as np

training = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    bag = [1 if w in pattern_words else 0 for w in words]

    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])


## MODEL

In [9]:
model = models.Sequential([
        layers.Input(shape=(len(train_x[0]),), name='input_layer'),
        layers.Dense(256, activation='relu', kernel_regularizer='l2', name='hidden_layer1'),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name='dropout1'),
        layers.Dense(128, activation='relu', kernel_regularizer='l2', name='hidden_layer2'),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name='dropout2'),
        layers.Dense(len(train_y[0]), activation='softmax', name='output_layer')
    ])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden_layer1 (Dense)                │ (None, 256)                 │         138,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout1 (Dropout)                   │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer2 (Dense)                │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout2 (Dropout)                   │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 81)                  │          10,449 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 183,633 (717.32 KB)

 Trainable params: 182,865 (714.32 KB)

 Non-trainable params: 768 (3.00 KB)

## K_FOLD

In [11]:
tf.compat.v1.reset_default_graph()

# Hyperparameters
num_folds = 5
batch_size = 8
learning_rate = 1e-4
epochs = 100


train_x = np.array(train_x)
train_y = np.array(train_y)


kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    print(f"\n=== Training Fold {fold + 1}/{num_folds} ===")
    
    X_train, X_val = train_x[train_index], train_x[val_index]
    y_train, y_val = train_y[train_index], train_y[val_index]

    tensorboard_callback = callbacks.TensorBoard(log_dir=f'./logs/fold_{fold+1}')
    early_stopping_callback = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[tensorboard_callback, early_stopping_callback],
        verbose=1
    )

    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} - Validation Accuracy: {val_accuracy:.4f}")
    fold_accuracies.append(val_accuracy)
    
avg_accuracy = np.mean(fold_accuracies)
print(f"\n=== Average Validation Accuracy: {avg_accuracy:.4f} ===")



=== Training Fold 1/5 ===
Epoch 1/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0066 - loss: 10.1489 - val_accuracy: 0.0153 - val_loss: 8.7074
Epoch 2/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0122 - loss: 8.9984 - val_accuracy: 0.0275 - val_loss: 8.2094
Epoch 3/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0364 - loss: 8.3259 - val_accuracy: 0.1101 - val_loss: 7.6646
Epoch 4/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0655 - loss: 7.7250 - val_accuracy: 0.1560 - val_loss: 7.1797
Epoch 5/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1068 - loss: 7.4008 - val_accuracy: 0.2141 - val_loss: 6.7905
Epoch 6/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1591 - loss: 6.9389 - val_accuracy: 0.2569 - val_loss: 6.4760
Epoch 7/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2093 - loss: 6.5206 - val_accuracy: 0.3028 - val_loss: 6.1926
Epoch 8/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2

## Callbacks

In [12]:
early_stopping_callback = callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
tensorboard_callback = callbacks.TensorBoard(log_dir='./logs/full_training')

## Training tanpa split

In [13]:
from sklearn.utils import shuffle
train_x, train_y = shuffle(train_x, train_y, random_state=42)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train and save the final model
model.fit(train_x, train_y, epochs=100, batch_size=8, callbacks=[tensorboard_callback, early_stopping_callback], verbose=1)
model.save('final_model_trained_on_full_data.h5')

Epoch 1/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8774 - loss: 0.7008
Epoch 2/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8915 - loss: 0.6509
Epoch 3/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9106 - loss: 0.5823
Epoch 4/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9260 - loss: 0.5465
Epoch 5/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9217 - loss: 0.5386
Epoch 6/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9315 - loss: 0.5000
Epoch 7/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9412 - loss: 0.4758
Epoch 8/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9617 - loss: 0.4461
Epoch 9/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9550 - loss: 0.4466
Epoch 10/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9582 - loss: 0.4273
Epoch 11/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9699 - loss: 0.3997
Epoch 12/100
205/205 ━━━━━━━━━━━━━━━━━━━━

In [14]:
import json

# Save
try:
    with open("training_data.json", "w") as file:
        # Convert non-serializable data like NumPy arrays or other objects to lists
        data = {'words': words, 'classes': classes, 'train_x': train_x.tolist(), 'train_y': train_y.tolist()}
        json.dump(data, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [15]:
# Save
try:
    with open("training_data", "wb") as file:
        pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [16]:
# Load
try:
    with open("training_data", "rb") as file:
        data = pickle.load(file)

    words = data['words']
    classes = data['classes']
    train_x = np.array(data['train_x'])
    train_y = np.array(data['train_y'])
except IOError as e:
    print(f"Error loading training data: {e}")

# Load model
try:
    model = keras.models.load_model('final_model_trained_on_full_data.h5')
except IOError as e:
    print(f"Error loading model: {e}")


In [17]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
            if show_details:
                print(f"Found in bag: {w}")
    return bag

ERROR_THRESHOLD = 0.40



def classify(sentence):
    input_data = bow(sentence, words)
    results = model.predict(np.array([input_data]))[0]
    results = [[i, r] for i, r in enumerate(results) if r >= ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        for intent in intents:
            if intent['tag'] == results[0][0]:
                return random.choice(intent['responses'])
    return "Maaf, saya belum mengerti apa yang Anda bicarakan, perlu bantuan?."

In [ ]:
print("0 to close")
while True:
    message = input("")
    if message == "0":
        break
    result = response(message)

    if result is not None and "~" in result:
        order = (result[1:])
        action(order)
    else:
        print(result)


0 to close


 halo


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Hallo! Apa kabar? Ada yang bisa saya bantu?


 bagaimana cara mengatasi overthinking?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Alihkan perhatianmu dengan aktivitas seperti meditasi, olahraga, atau hobi yang kamu nikmati.


 saya sedang mengalami overthinking


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Berikan waktu untuk dirimu sendiri tanpa tekanan dan praktikkan mindfulness untuk tetap tenang.


In [ ]:
model = tf.keras.models.load_model('final_model_trained_on_full_data.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open('model.tflite', 'wb') as f:
  f.write(tflite_model)